In [ ]:
# Importação:

import pandas as pd
from topsoft.database import configure_database

In [ ]:
# Cria o banco de dados e as tabelas necessárias (se ainda não existirem)

configure_database()

DADOS DOS ALUNOS


In [ ]:
# Lendo arquivo de alunos (exportado pelo TopSoft) com os dados já recebidos da API da ActivitySoft:

# TODO: Force each column to be a specific type
df_activity = pd.read_json(
    "alunos.json",
    dtype={
        "cartao_acesso": "string",
        "celular": "string",
        "cpf": "string",
        "data_nascimento": "date",
        "email": "string",
        "filiacao_1_id": "int",
        "filiacao_2_id": "int",
        "foto_data_hora_alteracao": "date",
        "id": "int",
        "id_turmas": "string",
        "matricula": "string",
        "nome": "string",
        "responsaveis_adicionais_ids": "string",
        "responsavel_id": "int",
        "responsavel_secundario_id": "int",
        "sexo": "string",
        "tipo_liberacao": "string",
        "unidade_id": "int",
        "url_foto": "string",
    },
)

# Drop column id to not conflict with the database primary key:
df_activity.drop(columns=["id"], inplace=True, errors="ignore")

# Formatando os nomes para garantir que não excedam 40 caracteres e removendo espaços extras:
df_activity["nome"] = df_activity["nome"].apply(lambda x: x[0:40].strip())


# Strip all string columns to remove leading/trailing spaces:
def strip_string_columns(df):
    for col in df.select_dtypes(include=["object"]).columns:
        df[col] = df[col].str.strip()
    return df


df_activity = strip_string_columns(df_activity)

# Forçando as colunas de data para o formato correto:
df_activity["data_nascimento"] = pd.to_datetime(
    df_activity["data_nascimento"], errors="coerce"
).dt.date
df_activity["foto_data_hora_alteracao"] = pd.to_datetime(
    df_activity["foto_data_hora_alteracao"], errors="coerce"
).dt.date

# Ordenando o DataFrame por nome:
df_activity.sort_values(by="nome", inplace=True)
display(df_activity)

In [ ]:
# UMA POSSÍVEL SOLUÇÃO PARA LER OS DADOS DO ALUNO DIRETAMENTE DO BANCO DE DADOS

# import sqlite3

# conn = sqlite3.connect('meu_banco.db')
# df_activity = pd.read_sql_table('minha_tabela', conn)

In [ ]:
# Validando se não existem nomes duplicados:

if df_activity["nome"].duplicated().any():
    print("Existem nomes duplicados no DataFrame de alunos.")
    display(df_activity[df_activity["nome"].duplicated(keep=False)])

ARQUIVO BILHETES.TXT CONTENDO NUMERO DA CARTEIRINHA + NOME


In [ ]:
# Lendo arquivo de carteirinhas (algum já pre-existente ou exportado pelo TopSoft):

with open(r"gi5_cartoes.txt", "r") as file:
    dados = file.read()

linhas = dados.split("\n")
carteirinhas = [{"cartao": i[0:16], "nome": i[16 : 16 + 40]} for i in linhas]

# Criando DataFrame de carteirinhas:
df_cartoes = pd.DataFrame(carteirinhas)

# Formatando os nomes para garantir que não excedam 40 caracteres e removendo espaços extras:
df_cartoes["nome"] = df_cartoes["nome"].apply(lambda x: x[0:40].strip())

# Drop empty names
df_cartoes = df_cartoes[df_cartoes["nome"] != ""]

# Ordenando o DataFrame por nome:
df_cartoes.sort_values(by="nome", inplace=True)
display(df_cartoes)

In [ ]:
# Validando se não existem nomes duplicados:


if df_cartoes["nome"].duplicated().any():


    print("Existem nomes duplicados no DataFrame de carteirinhas.")

    display(df_cartoes[df_cartoes["nome"].duplicated(keep=False)])

PRIMEIRO MERGE: TENTANDO VINCULAR CADA CARTEIRINHA A UM ALUNO


In [ ]:
# Merge dos DataFrames para vincular cada carteirinha ao aluno correspondente (baseado no nome):

df_merged = pd.merge(df_activity, df_cartoes, left_on="nome", right_on="nome")
display(df_merged[["nome", "cartao", "matricula"]])

In [ ]:
# Vamos atualizar o Banco de Dados local com os dados do DataFrame:

from topsoft.repository import (
    bind_cartao_acesso_to_aluno,
    get_or_create_cartao_acesso,
    get_or_create_aluno,
)

for index, row in df_merged.iterrows():
    cartao_acesso = get_or_create_cartao_acesso(row["cartao"])

    # Convert row to dict and remove NaN values
    dict_row = row.to_dict()
    dict_row = {k: v for k, v in dict_row.items() if pd.notna(v)}
    dict_row = {k: v for k, v in dict_row.items() if v is not None}
    # TODO: We could drop NaN values directly in the DataFrame before iterating, but this is a quick fix.

    # Create or get the aluno, passing all row values as keyword arguments
    aluno = get_or_create_aluno(**dict_row)

    bind_cartao_acesso_to_aluno(
        cartao_acesso_id=cartao_acesso.id,
        aluno_id=aluno.id,
    )
    print(f"Cartão {cartao_acesso.numeracao} vinculado ao aluno {aluno.nome}.")

SEGUNDO MERGE: CRIAR SUPOSTOS ALUNOS (POSSIVELMENTE FUNCIONÁRIOS) QUE ESTÃO PRESENTES NOS DADOS DA CATRACAS, MAS POR ALGUM MOTIVO NÃO ESTÃO NA LISTA COM OS DADOS DOS ALUNOS.


In [ ]:
# Também será necessário criar "alunos" para as carteirinhas que não possuem aluno correspondente (ainda), possivelmente é um funcionário...

not_aluno_df = df_cartoes[~df_cartoes["nome"].isin(df_activity["nome"])]
not_aluno_df = not_aluno_df.copy()
not_aluno_df["matricula"] = not_aluno_df["cartao"].apply(lambda x: f"cartao_{x}")
display(not_aluno_df[["nome", "cartao", "matricula"]])

In [ ]:
# Vamos atualizar o Banco de Dados local com os dados do DataFrame:

from topsoft.repository import (
    bind_cartao_acesso_to_aluno,
    get_or_create_cartao_acesso,
    get_or_create_aluno,
)

for index, row in not_aluno_df.iterrows():
    cartao_acesso = get_or_create_cartao_acesso(row["cartao"])

    # Convert row to dict and remove NaN values
    dict_row = row.to_dict()
    dict_row = {k: v for k, v in dict_row.items() if pd.notna(v)}
    dict_row = {k: v for k, v in dict_row.items() if v is not None}
    # TODO: We could drop NaN values directly in the DataFrame before iterating, but this is a quick fix.

    # Create or get the aluno, passing all row values as keyword arguments
    aluno = get_or_create_aluno(**dict_row)

    # Bulk update
    bind_cartao_acesso_to_aluno(
        cartao_acesso_id=cartao_acesso.id,
        aluno_id=aluno.id,
    )
    print(f"Cartão {cartao_acesso.numeracao} vinculado ao aluno {aluno.nome}.")